# 🪵 Mastering Python Logging: From Basics to Enterprise Patterns

**Welcome!** This notebook is your comprehensive guide to understanding and effectively using Python's built-in `logging` module. We'll go beyond simple `print()` statements to explore how robust logging is essential for building maintainable, diagnosable, and production-ready applications.

**Target Audience:** Python developers who want to implement professional-grade logging in their applications.

**Learning Objectives:**
*   Understand the *why* behind logging (debugging, monitoring, auditing).
*   Learn the different logging levels and when to use them.
*   Configure logging using `basicConfig`, `dictConfig`, Handlers, Formatters, and Filters.
*   Implement structured logging (e.g., JSON) for better machine readability.
*   Log exceptions effectively with tracebacks.
*   Explore best practices for logging in modular applications and enterprise environments.
*   Identify common pitfalls and prepare for related interview questions.

## 1. Introduction: Why Logging? (Beyond `print()`)

Think of logging as the **flight data recorder (black box)** for your application. While `print()` statements are useful for quick checks during development, they have major drawbacks in larger applications or production environments:

*   **No Severity Levels:** You can't easily distinguish between informational messages, warnings, and critical errors.
*   **Hard to Control:** Enabling/disabling `print` statements requires code changes.
*   **No Standard Format:** Output is inconsistent and hard to parse automatically.
*   **Performance:** Excessive printing can impact performance.
*   **Destination Flexibility:** `print` only goes to standard output; logging can go to files, network sockets, external services, etc.

**Logging provides solutions:**

1.  **Debugging & Diagnosis:** Track the flow of execution and variable states to pinpoint issues.
2.  **Monitoring:** Observe application health and performance in real-time.
3.  **Auditing:** Record significant events (e.g., user actions, configuration changes) for security or compliance.
4.  **Analysis:** Collect structured log data for later analysis (e.g., identifying usage patterns, performance bottlenecks).

**Use Case Analogy:** Imagine building a complex web server. Using `print` is like shouting occasional updates into a noisy room. Using `logging` is like having dedicated channels (levels), structured reports (formatters), and multiple destinations (files for errors, console for warnings, a central dashboard for critical alerts) managed by a central dispatcher (the logging framework).

## 2. The Basics: Levels and `basicConfig`

### 2.1 Logging Levels

The `logging` module defines standard severity levels. When you configure logging, you set a minimum level; messages with that severity or higher will be processed.

| Level      | Value | When to Use                                       |
| :--------- | ----: | :------------------------------------------------ |
| `DEBUG`    | 10    | Detailed information, typically only for diagnosis. |
| `INFO`     | 20    | Confirmation that things are working as expected.   |
| `WARNING`  | 30    | An indication of something unexpected or problematic in the near future (e.g., low disk space). The software is still working as expected. **(Default Level)** |
| `ERROR`    | 40    | Due to a more serious problem, the software has not been able to perform some function. |
| `CRITICAL` | 50    | A serious error, indicating that the program itself may be unable to continue running. |


In [1]:
import logging

# Default configuration only shows WARNING and above
print("--- Default Logging --- ")
logging.debug('This is a debug message (hidden by default)')
logging.info('This is an info message (hidden by default)')
logging.warning('This is a warning message (visible)')
logging.error('This is an error message (visible)')
logging.critical('This is a critical message (visible)')

ERROR:root:This is an error message (visible)
CRITICAL:root:This is a critical message (visible)


--- Default Logging --- 


### 2.2 Basic Configuration (`logging.basicConfig`)

`basicConfig` is a convenience function for simple configuration (e.g., scripts, small projects). **Important:** It only works if the root logger has not already been configured (e.g., by calling `logging.info` *before* `basicConfig`). Call it **once** at the very beginning of your application.

Common `basicConfig` arguments:
*   `level`: The minimum severity level to log (e.g., `logging.DEBUG`).
*   `format`: A string defining the output format (see below).
*   `datefmt`: A string defining the date/time format (used with `%(asctime)s` in the format string).
*   `filename`: If provided, logs output to this file instead of the console (stderr).
*   `filemode`: If `filename` is used, specifies the file mode ('a' for append (default), 'w' for write/overwrite).
*   `force`: (Python 3.8+) If True, removes and replaces existing handlers on the root logger. Useful in some testing or interactive scenarios, but use with caution in applications.

In [2]:
import logging
import sys # To ensure output is fresh in notebooks if re-run

# --- Configuration 1: Log DEBUG and above to console with a specific format ---
log_format = '%(asctime)s - %(levelname)s - %(name)s - %(filename)s:%(lineno)d - %(message)s'
date_format = '%Y-%m-%d %H:%M:%S'

# Use force=True in notebooks/interactive sessions if you might re-run the cell
# In a standard application, avoid 'force=True' unless you have a specific reason.
logging.basicConfig(level=logging.DEBUG, 
                    format=log_format, 
                    datefmt=date_format,
                    stream=sys.stderr, # Explicitly set stream for notebooks
                    force=True) 

print("\n--- Custom BasicConfig Logging --- ")
logging.debug('This is a detailed debug message.')
logging.info('Application starting up.')
logging.warning('Configuration file not found, using defaults.')
logging.error('Failed to connect to database.')
logging.critical('System meltdown imminent!')

# --- Configuration 2: Log INFO and above to a file (overwriting) ---
# Note: Running this will change the logging destination for subsequent calls in this session
# logging.basicConfig(level=logging.INFO, 
#                     format=log_format, 
#                     datefmt=date_format,
#                     filename='app.log', 
#                     filemode='w',
#                     force=True)
# logging.info('This message goes to the file app.log')

2025-04-20 16:13:40 - DEBUG - root - 464591755.py:17 - This is a detailed debug message.
2025-04-20 16:13:40 - INFO - root - 464591755.py:18 - Application starting up.
2025-04-20 16:13:40 - WARNING - root - 464591755.py:19 - Configuration file not found, using defaults.
2025-04-20 16:13:40 - ERROR - root - 464591755.py:20 - Failed to connect to database.
2025-04-20 16:13:40 - CRITICAL - root - 464591755.py:21 - System meltdown imminent!



--- Custom BasicConfig Logging --- 


**Common LogRecord Attributes for Formatting:**

| Attribute      | Format                  | Description                                                 |
| :------------- | :---------------------- | :---------------------------------------------------------- |
| `asctime`      | `%(asctime)s`           | Human-readable time when the `LogRecord` was created.       |
| `created`      | `%(created)f`           | Time when the `LogRecord` was created (Unix timestamp).     |
| `filename`     | `%(filename)s`          | Filename portion of pathname.                               |
| `funcName`     | `%(funcName)s`          | Name of function containing the logging call.               |
| `levelname`    | `%(levelname)s`         | Text logging level for the message ('DEBUG', 'INFO', etc.). |
| `levelno`      | `%(levelno)s`           | Numeric logging level for the message (10, 20, etc.).       |
| `lineno`       | `%(lineno)d`            | Source line number where the logging call was issued.       |
| `message`      | `%(message)s`           | The logged message itself.                                  |
| `module`       | `%(module)s`            | Module name.                                                |
| `msecs`        | `%(msecs)d`             | Millisecond portion of the creation time.                   |
| `name`         | `%(name)s`              | Name of the logger used to log the call.                    |
| `pathname`     | `%(pathname)s`          | Full pathname of the source file.                           |
| `process`      | `%(process)d`           | Process ID (if available).                                  |
| `processName`  | `%(processName)s`       | Process name (if available).                                |
| `thread`       | `%(thread)d`            | Thread ID (if available).                                   |
| `threadName`   | `%(threadName)s`        | Thread name (if available).                                 |

Full list: [LogRecord attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes)

## 3. Loggers, Handlers, Formatters, and Filters: The Core Components

`basicConfig` is limited. For more control (e.g., different formats for console vs. file, logging specific modules differently), you need to work with the core components:

1.  **Loggers:** Your application code interacts directly with Logger objects (`logging.getLogger(...)`). They are arranged in a hierarchy (e.g., `myapp.network.utils` is a child of `myapp.network`, which is a child of `myapp`, which is a child of the `root` logger). Loggers decide *whether* to process a message based on their level.
2.  **Handlers:** Take the log records from loggers and dispatch them to the appropriate destination (console, file, network socket, email, etc.). Handlers also have levels; a handler will only process messages at its level or higher.
3.  **Formatters:** Specify the layout of the final log message string.
4.  **Filters:** Provide fine-grained control over which log records are passed from loggers to handlers or from handlers onwards.

**Flow:** Log message -> Logger (checks level) -> Filters (on Logger) -> Handlers (check level) -> Filters (on Handler) -> Formatter -> Destination

### 3.1 Getting a Logger (Best Practice: `__name__`)

It's standard practice to get a logger specific to the current module using `logging.getLogger(__name__)`. This automatically uses the module's dotted path as the logger name, fitting it into the hierarchy and making it easy to configure logging per module.


In [3]:
# In your_module.py
import logging

# Get a logger named after the module (e.g., 'your_module')
logger = logging.getLogger(__name__) 

logger.info(f"This log message comes from the '{logger.name}' logger.")

# In another_module.py (assuming it's in a package 'mypackage')
# import logging
# logger = logging.getLogger(__name__) # logger name will be 'mypackage.another_module'
# logger.debug("Debug info from another module.")

2025-04-20 16:13:40 - INFO - __main__ - 3570662048.py:7 - This log message comes from the '__main__' logger.


### 3.2 Configuring Handlers and Formatters

Let's manually configure a logger with two handlers: one for the console (showing INFO and above) and one for a file (showing ERROR and above).

In [4]:
import logging
import sys

# 1. Get the logger (use __name__ or a specific name)
module_logger = logging.getLogger('my_app_module')
module_logger.setLevel(logging.DEBUG) # Process all messages from DEBUG upwards

# Prevent messages from propagating to the root logger's handlers
# Set this to False if you *only* want this logger's handlers to act
module_logger.propagate = False 

# 2. Create Handlers
# Console Handler (INFO and above)
console_handler = logging.StreamHandler(sys.stderr) # Use stderr for logs by convention
console_handler.setLevel(logging.INFO) 

# File Handler (ERROR and above)
file_handler = logging.FileHandler('error.log', mode='a') # Append mode
file_handler.setLevel(logging.ERROR)

# 3. Create Formatters
console_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%H:%M:%S')
file_format = logging.Formatter('%(asctime)s - %(process)d - %(thread)d - %(name)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s')

# 4. Set Formatters on Handlers
console_handler.setFormatter(console_format)
file_handler.setFormatter(file_format)

# 5. Add Handlers to the Logger
# Ensure handlers aren't added multiple times if re-running cell in notebook
if not module_logger.handlers:
    module_logger.addHandler(console_handler)
    module_logger.addHandler(file_handler)
else:
    # In a real app, you configure once. In notebooks, clear existing if re-running:
    module_logger.handlers.clear()
    module_logger.addHandler(console_handler)
    module_logger.addHandler(file_handler)

# 6. Log Messages
print("\n--- Manual Handler Configuration --- ")
module_logger.debug("This debug message won't appear on console or file.")
module_logger.info("Application finished processing request.") # Appears on console
module_logger.warning("User authentication failed for user 'admin'.") # Appears on console
module_logger.error("Database connection lost.") # Appears on console AND in error.log
module_logger.critical("System integrity compromised!") # Appears on console AND in error.log

16:13:40 - my_app_module - INFO - Application finished processing request.
16:13:40 - my_app_module - WARNING - User authentication failed for user 'admin'.
16:13:40 - my_app_module - ERROR - Database connection lost.
16:13:40 - my_app_module - CRITICAL - System integrity compromised!



--- Manual Handler Configuration --- 


### 3.3 Logger Hierarchy and Propagation

Loggers exist in a hierarchy based on their names (e.g., `a.b` is a child of `a`). By default, after a logger processes a message and sends it to its *own* handlers, it passes the message up to its parent's handlers. This continues up to the root logger.

*   You can disable this by setting `logger.propagate = False`.
*   This allows you to configure a high-level logger (e.g., `myapp`) and have all child loggers (`myapp.ui`, `myapp.db`) inherit that configuration unless they have specific handlers or propagation disabled.

**Pitfall:** If you see duplicate log messages, it's often because a logger *and* one of its ancestors both have handlers (like the root logger configured by `basicConfig`) and propagation is enabled (the default).

### 3.4 Filters

Filters provide more granular control than levels. You can filter based on logger name, level, or custom logic.

A filter can be added to a Logger or a Handler using `addFilter()`. The filter object must have a `filter(record)` method that returns `True` if the record should be processed, `False` otherwise.

In [5]:
import logging
import sys

# Filter to only allow records from a specific module/logger name
class ModuleFilter(logging.Filter):
    def __init__(self, module_name: str):
        super().__init__()
        self.module_name = module_name

    def filter(self, record: logging.LogRecord) -> bool:
        # Check if the record's logger name starts with the specified module name
        return record.name.startswith(self.module_name)

# --- Setup --- 
logger_a = logging.getLogger('myapp.network')
logger_b = logging.getLogger('myapp.database')
logger_a.setLevel(logging.INFO)
logger_b.setLevel(logging.INFO)

console_handler = logging.StreamHandler(sys.stderr)
console_handler.setLevel(logging.INFO)
console_format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(console_format)

# Add filter to the handler: Only show messages from 'myapp.network'
network_filter = ModuleFilter('myapp.network')
console_handler.addFilter(network_filter)

# Add handler to both loggers (normally might add to parent 'myapp')
# Clear handlers if re-running cell
for logger in [logger_a, logger_b]:
    if not logger.handlers:
        logger.addHandler(console_handler)
    # Ensure propagation is off if adding handler directly to multiple children
    # OR add the handler to the parent ('myapp') and let propagation work.
    logger.propagate = False 

# --- Logging --- 
print("\n--- Filter Demonstration --- ")
logger_a.info("Network connection established.") # Should appear (passes filter)
logger_b.info("Database query executed.")      # Should NOT appear (filtered out)
logger_a.warning("High latency detected.")     # Should appear (passes filter)

# Cleanup: Remove filter and handlers for subsequent cells if needed
console_handler.removeFilter(network_filter)
logger_a.removeHandler(console_handler)
logger_b.removeHandler(console_handler)

myapp.network - INFO - Network connection established.
myapp.network - WARNING - High latency detected.



--- Filter Demonstration --- 


## 4. Advanced Configuration: `fileConfig` and `dictConfig`

Manually creating handlers, formatters, etc., in code can become verbose. For complex applications, configuration is often externalized.

*   **`logging.config.fileConfig()`:** Reads configuration from a file in a specific format (similar to `.ini` files). Less common now due to its inflexibility compared to `dictConfig`.
*   **`logging.config.dictConfig()`:** Reads configuration from a Python dictionary. This is the **recommended modern approach** as dictionaries are easy to create, modify programmatically, load from JSON/YAML, and offer more flexibility.


### 4.1 `dictConfig` Example

This dictionary defines loggers, handlers, and formatters structurally.

In [6]:
import logging
import logging.config
import sys

LOGGING_CONFIG = {
    'version': 1, # Schema version
    'disable_existing_loggers': False, # Keep loggers configured by libraries
    
    # Define formatters
    'formatters': {
        'simple': {
            'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            'datefmt': '%H:%M:%S'
        },
        'detailed': {
            'format': '%(asctime)s - %(name)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(process)d - %(thread)d - %(message)s'
        },
    },
    
    # Define handlers
    'handlers': {
        'console': {
            'class': 'logging.StreamHandler',
            'level': 'DEBUG', # Handler level
            'formatter': 'simple', # Use the 'simple' formatter
            'stream': 'ext://sys.stderr' # Direct output to stderr
        },
        'file_app': {
            'class': 'logging.FileHandler',
            'level': 'INFO',
            'formatter': 'detailed',
            'filename': 'application.log', # Log file name
            'mode': 'a', # Append mode
            'encoding': 'utf-8'
        },
        'file_errors': {
            'class': 'logging.FileHandler',
            'level': 'ERROR',
            'formatter': 'detailed',
            'filename': 'errors.log',
            'mode': 'a',
            'encoding': 'utf-8'
        },
    },
    
    # Define loggers
    'loggers': {
        'myapp': { # Configures logger named 'myapp' and its children
            'level': 'INFO', # Logger level
            'handlers': ['console', 'file_app', 'file_errors'], # Attach handlers
            'propagate': False # Don't pass to root logger handlers
        },
        'third_party_lib': { # Example: Quieter logging for a library
            'level': 'WARNING',
            'handlers': ['console'],
            'propagate': False
        }
    },
    
    # Configure the root logger (optional, handles anything not caught by specific loggers)
    'root': {
        'level': 'WARNING',
        'handlers': ['console']
    }
}

# Apply the configuration
logging.config.dictConfig(LOGGING_CONFIG)

# Get loggers (names defined in the config)
app_logger = logging.getLogger('myapp.service') # Child of 'myapp'
lib_logger = logging.getLogger('third_party_lib')
root_logger_test = logging.getLogger('unconfigured_logger') # Handled by root

# Log some messages
print("\n--- dictConfig Logging --- ")
app_logger.debug("Detailed service debug info.") # Filtered by logger level 'INFO'
app_logger.info("Service started successfully.") # Goes to console, file_app
app_logger.warning("Cache miss for resource X.") # Goes to console, file_app
app_logger.error("Failed to process message ID 123.") # Goes to console, file_app, file_errors
lib_logger.info("Some info from the library.") # Filtered by logger level 'WARNING'
lib_logger.warning("Library component deprecated.") # Goes to console
root_logger_test.warning("Warning from an unconfigured logger.") # Handled by root -> console

16:13:40 - myapp.service - INFO - Service started successfully.
16:13:40 - myapp.service - WARNING - Cache miss for resource X.
16:13:40 - myapp.service - ERROR - Failed to process message ID 123.
16:13:40 - third_party_lib - WARNING - Library component deprecated.
16:13:40 - unconfigured_logger - WARNING - Warning from an unconfigured logger.



--- dictConfig Logging --- 


## 5. Structured Logging (e.g., JSON)

**Why?** In modern systems (microservices, cloud), logs are often aggregated into centralized platforms (Elasticsearch/Logstash/Kibana - ELK, Splunk, Datadog). Plain text logs are hard for machines to parse reliably. Structured logs (like JSON) make querying, filtering, and analysis much easier.

**How?** Use custom formatters or dedicated libraries.

### 5.1 Using `python-json-logger`

A popular library for creating JSON logs.

`pip install python-json-logger`

In [7]:
# Install if you don't have it
# !pip install python-json-logger

import logging
import logging.config
from pythonjsonlogger import jsonlogger
import datetime
import sys

# Define a custom JSON formatter
# See library docs for more formatting options
class CustomJsonFormatter(jsonlogger.JsonFormatter):
    def add_fields(self, log_record, record, message_dict):
        super(CustomJsonFormatter, self).add_fields(log_record, record, message_dict)
        if not log_record.get('timestamp'):
            now = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.%fZ')
            log_record['timestamp'] = now
        if log_record.get('level'):
            log_record['level'] = log_record['level'].upper()
        else:
            log_record['level'] = record.levelname

JSON_LOGGING_CONFIG = {
    'version': 1,
    'disable_existing_loggers': False,
    'formatters': {
        'json': {
            # Reference the custom class using its full path
            '()': '__main__.CustomJsonFormatter', # Use '__main__' in notebooks, or the actual module path
            'format': '%(timestamp)s %(level)s %(name)s %(message)s %(filename)s %(lineno)d'
        }
    },
    'handlers': {
        'json_console': {
            'class': 'logging.StreamHandler',
            'level': 'INFO',
            'formatter': 'json',
            'stream': 'ext://sys.stderr'
        }
    },
    'loggers': {
        'json_app': {
            'level': 'INFO',
            'handlers': ['json_console'],
            'propagate': False
        }
    }
}

logging.config.dictConfig(JSON_LOGGING_CONFIG)

json_logger = logging.getLogger('json_app.worker')

print("\n--- JSON Structured Logging --- ")
# Logging simple messages
json_logger.info("Worker process started.")

# Logging with extra context (becomes part of the JSON object)
extra_data = {
    'user_id': 'user-123',
    'request_id': 'req-abc-987',
    'elapsed_ms': 55.6
}
json_logger.info("Processed task successfully.", extra={'extra_data': extra_data})
json_logger.error("Failed to update record.", extra={'record_id': 456, 'reason': 'DB timeout'})

/tmp/ipykernel_9931/2235547257.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.%fZ')
{"timestamp": "2025-04-20T10:43:41.031386Z", "level": "INFO", "name": "json_app.worker", "message": "Worker process started.", "filename": "2235547257.py", "lineno": 56}
{"timestamp": "2025-04-20T10:43:41.032850Z", "level": "INFO", "name": "json_app.worker", "message": "Processed task successfully.", "filename": "2235547257.py", "lineno": 64, "extra_data": {"user_id": "user-123", "request_id": "req-abc-987", "elapsed_ms": 55.6}}
{"timestamp": "2025-04-20T10:43:41.034012Z", "level": "ERROR", "name": "json_app.worker", "message": "Failed to update record.", "filename": "2235547257.py", "lineno": 65, "record_id": 456, "reason": "DB timeout"}



--- JSON Structured Logging --- 


**Note:** Libraries like `structlog` offer even more advanced features for structured logging, including processors for adding context automatically.

## 6. Logging Exceptions

It's crucial to log the full traceback when an exception occurs to understand the context of the error.

*   **`logger.exception(msg, *args, **kwargs)`:** Logs a message with level ERROR and automatically includes the current exception information (traceback).
*   **`logger.error(msg, *args, exc_info=True, **kwargs)`:** Logs a message with level ERROR and explicitly requests the exception information to be included.

Use these inside an `except` block.

In [8]:
import logging

# Assuming basicConfig or dictConfig has been run previously
# If not, uncomment the basicConfig line:
# logging.basicConfig(level=logging.ERROR, force=True, stream=sys.stderr)
logger = logging.getLogger('exception_demo')
logger.propagate = True # Ensure it goes to root handler if basicConfig was used

print("\n--- Exception Logging --- ")
try:
    result = 10 / 0
except ZeroDivisionError as e:
    # Method 1: logger.exception (preferred for brevity)
    logger.exception(f"Calculation failed due to division by zero. Original error: {e}")

try:
    my_list = [1, 2]
    val = my_list[5]
except IndexError as e:
    # Method 2: logger.error with exc_info=True
    logger.error(f"Index out of bounds accessing list. Error: {e}", exc_info=True)

try:
    x = int('abc')
except ValueError:
    # Example of logging without automatic traceback (less ideal for exceptions)
    logger.error("Could not convert string to int. Manual traceback logging needed if required.")


--- Exception Logging --- 


16:13:41 - exception_demo - ERROR - Calculation failed due to division by zero. Original error: division by zero
Traceback (most recent call last):
  File "/tmp/ipykernel_9931/2592193953.py", line 11, in <module>
    result = 10 / 0
             ~~~^~~
ZeroDivisionError: division by zero
16:13:41 - exception_demo - ERROR - Index out of bounds accessing list. Error: list index out of range
Traceback (most recent call last):
  File "/tmp/ipykernel_9931/2592193953.py", line 18, in <module>
    val = my_list[5]
          ~~~~~~~^^^
IndexError: list index out of range
16:13:41 - exception_demo - ERROR - Could not convert string to int. Manual traceback logging needed if required.


## 7. Useful Handlers

Beyond `StreamHandler` and `FileHandler`:

*   **`logging.handlers.RotatingFileHandler`:** Rotates log files when they reach a certain size (`maxBytes`). Keeps a specified number of backup files (`backupCount`). Essential for preventing log files from filling up disk space.
*   **`logging.handlers.TimedRotatingFileHandler`:** Rotates log files at specific time intervals (`when`, `interval`, e.g., daily, hourly). Also keeps backups (`backupCount`). Useful for archiving logs based on time.
*   **`logging.handlers.SocketHandler` / `DatagramHandler`:** Sends log records over TCP or UDP, respectively (e.g., to a central log server).
*   **`logging.handlers.SysLogHandler`:** Sends logs to a Unix/Linux syslog daemon.
*   **`logging.handlers.NTEventLogHandler`:** Sends logs to the Windows Event Log.
*   **`logging.handlers.SMTPHandler`:** Sends critical logs via email.
*   **`logging.handlers.QueueHandler` / `QueueListener`:** (Advanced) Useful for sending logs from multiple processes or threads to a central logging process without blocking the worker processes/threads.

In [9]:
import logging
import time
from logging.handlers import RotatingFileHandler, TimedRotatingFileHandler

# --- RotatingFileHandler Example ---
rotate_logger = logging.getLogger('rotator')
rotate_logger.setLevel(logging.INFO)
rotate_logger.propagate = False

# Rotate after 1KB (small for demo), keep 3 backup files (app.log, app.log.1, app.log.2)
rotate_handler = RotatingFileHandler('rotating_app.log', maxBytes=1024, backupCount=3, encoding='utf-8')
rotate_format = logging.Formatter('%(asctime)s - %(message)s')
rotate_handler.setFormatter(rotate_format)

if not rotate_logger.handlers:
    rotate_logger.addHandler(rotate_handler)

print("\n--- RotatingFileHandler Demo (logging to rotating_app.log) ---")
# Log enough data to trigger rotation
for i in range(20):
    rotate_logger.info(f"Log entry number {i+1} with some padding text to increase size.")
print("Check rotating_app.log and its backups (e.g., rotating_app.log.1)")

# --- TimedRotatingFileHandler Example ---
timed_logger = logging.getLogger('timer')
timed_logger.setLevel(logging.DEBUG)
timed_logger.propagate = False

# Rotate every 5 seconds (short for demo), keep 2 backups
# 'when' options: 's', 'm', 'h', 'd', 'midnight', 'w0'-'w6'
timed_handler = TimedRotatingFileHandler('timed_app.log', when='s', interval=5, backupCount=2, encoding='utf-8')
timed_format = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
timed_handler.setFormatter(timed_format)

if not timed_logger.handlers:
    timed_logger.addHandler(timed_handler)

# print("\n--- TimedRotatingFileHandler Demo (logging every second for ~12s) ---")
# for i in range(12):
#     timed_logger.debug(f"Timed log entry {i+1}")
#     time.sleep(1)
# print("Check timed_app.log and its timestamped backups (e.g., timed_app.log.YYYY-MM-DD_HH-MM-SS)")

# Clean up handlers to avoid interference if cell is re-run
rotate_logger.removeHandler(rotate_handler)
timed_logger.removeHandler(timed_handler)
rotate_handler.close()
timed_handler.close()


--- RotatingFileHandler Demo (logging to rotating_app.log) ---
Check rotating_app.log and its backups (e.g., rotating_app.log.1)


## 8. Enterprise Best Practices & Considerations

1.  **Use `getLogger(__name__)`:** Always get module-specific loggers.
2.  **Configure via `dictConfig` (or File):** Externalize configuration from code. Load from YAML/JSON for flexibility.
3.  **Use Structured Logging (JSON):** Essential for centralized logging systems (ELK, Splunk, Datadog, CloudWatch Logs, etc.). Makes logs searchable and analyzable.
4.  **Log Exceptions Correctly:** Use `logger.exception()` or `logger.error(..., exc_info=True)`.
5.  **Choose Appropriate Levels:** Don't overuse DEBUG in production. Log INFO for key workflow steps, WARNING for recoverable issues, ERROR/CRITICAL for serious problems.
6.  **Include Context:** Add relevant information (user IDs, request IDs, correlation IDs) to log messages (use `extra` dictionary or structured logging features). This is vital for tracing requests across services.
7.  **Avoid Logging Sensitive Data:** Never log passwords, API keys, personal information (PII), etc., unless absolutely necessary and properly secured/masked. Be mindful of GDPR and other privacy regulations.
8.  **Use Rotating Handlers:** Prevent logs from consuming all disk space (`RotatingFileHandler` or `TimedRotatingFileHandler`).
9.  **Performance:** Logging has overhead. Avoid excessive logging (especially DEBUG) in performance-critical code paths. Consider asynchronous logging (`QueueHandler`) if logging becomes a bottleneck.
10. **Centralized Logging:** In distributed systems (microservices), send logs from all services to a central platform for unified viewing and analysis.
11. **Consistency:** Establish logging conventions (format, levels, context fields) across your team or organization.

## 9. Pitfalls and Common Interview Questions

**Common Pitfalls:**

*   **Calling `basicConfig` too late:** It has no effect if a logger has already been configured.
*   **Duplicate Logs:** Usually caused by `propagate = True` (default) and handlers configured on both a child and ancestor logger.
*   **Forgetting `getLogger(__name__)`:** Using the root logger directly everywhere makes fine-grained control difficult.
*   **Logging Sensitive Information:** A major security risk.
*   **Excessive Logging:** Filling disks or incurring high costs in cloud logging services.
*   **Inconsistent Formatting:** Makes automated parsing difficult.
*   **Not Logging Tracebacks:** Makes debugging errors much harder.
*   **Blocking I/O:** Synchronous logging to slow destinations (network, busy disk) can block your application.

**Common Interview Questions:**

1.  Why use logging instead of `print()`?
2.  Explain the different logging levels.
3.  How do you configure logging in Python? (Mention `basicConfig`, `dictConfig`).
4.  What are Handlers, Formatters, and Filters?
5.  What is the purpose of `logging.getLogger(__name__)`?
6.  What is logger propagation?
7.  How would you log an exception with its traceback?
8.  What is structured logging (e.g., JSON logging) and why is it important?
9.  How can you prevent log files from becoming too large?
10. What are some best practices for logging in a production application?
11. How would you add contextual information (like a request ID) to all log messages for a specific request?
12. What security considerations are important when logging?

## 10. Challenge: Configure Logging for a Multi-Module App

**Scenario:** You have a simple application with two modules: `main_app.py` and `utils.py`.

**`utils.py`:**
```python
import logging

logger = logging.getLogger(__name__) # Should be 'utils'

def process_data(data):
    logger.debug(f"Processing data: {data!r}")
    if not isinstance(data, dict):
        logger.warning("Invalid data type received, expected dict.")
        return None
    result = data.get('value', 0) * 2
    logger.info("Data processed successfully.")
    return result
```

**`main_app.py`:**
```python
import logging
import logging.config
# Assume utils.py is in the same directory or sys.path
import utils 

# --- Your logging configuration dictionary here --- 
LOGGING_CONFIG = {
    'version': 1,
    'disable_existing_loggers': False,
    # ... Add formatters, handlers, loggers ... 
}

logging.config.dictConfig(LOGGING_CONFIG)

main_logger = logging.getLogger(__name__) # Should be '__main__'

def run_app():
    main_logger.info("Application starting.")
    result1 = utils.process_data({'value': 10})
    main_logger.debug(f"Result 1: {result1}")
    
    result2 = utils.process_data("bad data")
    main_logger.debug(f"Result 2: {result2}")
    
    try:
        x = 1 / 0
    except ZeroDivisionError:
        main_logger.exception("An error occurred during calculation.")
        
    main_logger.info("Application finished.")

if __name__ == "__main__":
    run_app()
```

**Task:**

1.  Create the `LOGGING_CONFIG` dictionary for `main_app.py` using `dictConfig`.
2.  Configure it to meet these requirements:
    *   Log `INFO` level and above from *all* loggers (`__main__`, `utils`) to the console using a simple format (`%(levelname)s:%(name)s:%(message)s`).
    *   Log `DEBUG` level and above *only* from the `utils` logger to a file named `utils_debug.log` using a detailed format (`%(asctime)s - %(name)s - %(levelname)s - %(message)s`).
    *   Log `ERROR` level and above from *all* loggers to a file named `app_errors.log` including detailed formatting and traceback info where applicable.
3.  Ensure logs are not duplicated unnecessarily.
4.  (Bonus) Modify the configuration to use JSON formatting for the console output.

*(Self-Correction Note: Think about logger levels vs handler levels and propagation when designing the config!)*

## 11. Conclusion

Effective logging is a cornerstone of professional software development. Moving beyond basic `print` statements to utilize Python's `logging` module provides structure, control, and invaluable insights into your application's behavior. By mastering levels, configuration (especially `dictConfig`), handlers, formatters, structured logging, and best practices, you can build applications that are significantly easier to debug, monitor, and maintain in any environment.

Remember to treat logging configuration as part of your application's architecture, adapting it as your application grows and evolves.